In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
train_df = pd.read_csv('../input/widsdatathon2021/TrainingWiDS2021.csv')
train_df.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,1

# Summarize the dataset

In [5]:
# dimension of the dataset
print(f'Train_df shape is: {train_df.shape}')

Train_df shape is: (130157, 181)


We have 130,157 observations with 181 features including target variable `diabetes_mellitus`

In [6]:
# peek at the dataset
print(f'Train_df statistical summary: \n{train_df.describe()}')

Train_df statistical summary: 
          Unnamed: 0   encounter_id    hospital_id            age  \
count  130157.000000  130157.000000  130157.000000  125169.000000   
mean    65079.000000  213000.856519     106.102131      61.995103   
std     37573.233831   38109.828146      63.482277      16.822880   
min         1.000000  147000.000000       1.000000       0.000000   
25%     32540.000000  180001.000000      49.000000      52.000000   
50%     65079.000000  213014.000000     112.000000      64.000000   
75%     97618.000000  246002.000000     165.000000      75.000000   
max    130157.000000  279000.000000     204.000000      89.000000   

                 bmi  elective_surgery         height         icu_id  \
count  125667.000000     130157.000000  128080.000000  130157.000000   
mean       29.110260          0.189840     169.607219     662.428344   
std         8.262776          0.392176      10.833085     304.259843   
min        14.844926          0.000000     137.200000      

In [7]:
# let's check the class distribution
print(f'Class distribution in train_df: \n{train_df.groupby("diabetes_mellitus").size()}')

Class distribution in train_df: 
diabetes_mellitus
0    102006
1     28151
dtype: int64


It is obvious that there is an imbalance in the target class with about 78.3% biased to people who are free of diabetes 

In [10]:
# check missing values in the train dataset
train_df.isna().sum().sum()

8018193

In [12]:
# investigating missing values per column sorted in Desc order in %
train_df.isnull().sum().sort_values(ascending = False)/len(train_df) * 100

h1_bilirubin_min               92.089553
h1_bilirubin_max               92.089553
h1_albumin_min                 91.431886
h1_albumin_max                 91.431886
h1_lactate_max                 91.018539
h1_lactate_min                 91.018539
h1_pao2fio2ratio_min           87.123243
h1_pao2fio2ratio_max           87.123243
h1_arterial_ph_min             82.860699
h1_arterial_ph_max             82.860699
h1_arterial_pco2_max           82.720100
h1_arterial_pco2_min           82.720100
h1_arterial_po2_min            82.550305
h1_arterial_po2_max            82.550305
h1_hco3_min                    81.743587
h1_hco3_max                    81.743587
h1_wbc_max                     81.429351
h1_wbc_min                     81.429351
h1_calcium_max                 81.379411
h1_calcium_min                 81.379411
h1_platelets_min               81.231897
h1_platelets_max               81.231897
h1_bun_min                     80.664121
h1_bun_max                     80.664121
h1_diasbp_invasi

In [14]:
# set a treshhold
treshold = 30
(train_df.isnull().sum().sort_values()/len(train_df) * 100) < treshold

Unnamed: 0                      True
lymphoma                        True
leukemia                        True
immunosuppression               True
hepatic_failure                 True
cirrhosis                       True
aids                            True
solid_tumor_with_metastasis     True
ventilated_apache               True
intubated_apache                True
arf_apache                      True
apache_post_operative           True
readmission_status              True
pre_icu_los_days                True
diabetes_mellitus               True
icu_stay_type                   True
icu_type                        True
encounter_id                    True
icu_id                          True
hospital_id                     True
elective_surgery                True
gender                          True
icu_admit_source                True
d1_heartrate_max                True
d1_heartrate_min                True
d1_sysbp_max                    True
d1_sysbp_min                    True
d

Investigating what type of missing values are 

* Zero Values
* Missing Values
* % of Total Values
* Total Zero Missing Values
* % Total Zero Missing Values
* Data Type

In [16]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [17]:
missing_zero_values_table(train_df)

Your selected dataframe has 181 columns and 130157 Rows.
There are 160 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
h1_bilirubin_min,0,119861,92.1,119861,92.1,float64
h1_bilirubin_max,0,119861,92.1,119861,92.1,float64
h1_albumin_max,0,119005,91.4,119005,91.4,float64
h1_albumin_min,0,119005,91.4,119005,91.4,float64
h1_lactate_max,0,118467,91.0,118467,91.0,float64
h1_lactate_min,0,118467,91.0,118467,91.0,float64
h1_pao2fio2ratio_min,0,113397,87.1,113397,87.1,float64
h1_pao2fio2ratio_max,0,113397,87.1,113397,87.1,float64
h1_arterial_ph_max,0,107849,82.9,107849,82.9,float64
h1_arterial_ph_min,0,107849,82.9,107849,82.9,float64


In [26]:
# Investigate categorical variables and how many levels each one has in one bug dictionary
temp = {str(k): list(v) for k, v in train_df.groupby(train_df.dtypes, axis=1)}
# [print(f'col {col}: {train_df[col].dtype}') for col in train_df]
temp

{'int64': ['Unnamed: 0',
  'encounter_id',
  'hospital_id',
  'elective_surgery',
  'icu_id',
  'readmission_status',
  'apache_post_operative',
  'arf_apache',
  'intubated_apache',
  'ventilated_apache',
  'aids',
  'cirrhosis',
  'hepatic_failure',
  'immunosuppression',
  'leukemia',
  'lymphoma',
  'solid_tumor_with_metastasis',
  'diabetes_mellitus'],
 'float64': ['age',
  'bmi',
  'height',
  'pre_icu_los_days',
  'weight',
  'albumin_apache',
  'apache_2_diagnosis',
  'apache_3j_diagnosis',
  'bilirubin_apache',
  'bun_apache',
  'creatinine_apache',
  'fio2_apache',
  'gcs_eyes_apache',
  'gcs_motor_apache',
  'gcs_unable_apache',
  'gcs_verbal_apache',
  'glucose_apache',
  'heart_rate_apache',
  'hematocrit_apache',
  'map_apache',
  'paco2_apache',
  'paco2_for_ph_apache',
  'pao2_apache',
  'ph_apache',
  'resprate_apache',
  'sodium_apache',
  'temp_apache',
  'urineoutput_apache',
  'wbc_apache',
  'd1_diasbp_invasive_max',
  'd1_diasbp_invasive_min',
  'd1_diasbp_max',
